In [34]:
import numpy as np
import tqdm
import random
import scipy.misc
import os
import numpy.random as rng
from sklearn.utils import shuffle
import tensorflow as tf
import keras
import keras.backend as K
import scipy
from keras.applications.vgg16 import VGG16
from keras.models import Sequential,Model
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten, MaxPooling2D
from keras.regularizers import l2
from keras import optimizers

In [46]:
##VGG16 augmented siamese network
w_init=keras.initializers.RandomNormal(mean=0.0, stddev=0.2, seed=10)
b_init=keras.initializers.RandomNormal(mean=0.5, stddev=0.01, seed=10)
input_shape = (64,64,3)
left_input = Input(input_shape)
right_input = Input(input_shape)
VGG=VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(64,64,3), pooling=None)
model = Sequential()
for l in VGG.layers:
    model.add(l)
model.add(Flatten(input_shape=VGG.output_shape[1:]))
model.add(Dense(4096, activation='sigmoid',kernel_regularizer=l2(0.01),kernel_initializer=w_init,bias_initializer=b_init))
first_twin = model(left_input)
second_twin = model(right_input)
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[19:20]:
    layer.trainable = True
L1_layer =Lambda(lambda x: K.abs(x[0]-x[1]))
L1_distance = L1_layer([first_twin, second_twin])
prediction = Dense(1,activation='sigmoid')(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
#optimizer=optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False )
optimizer = keras.optimizers.Adam(0.00006)
siamese_net.compile(optimizer=optimizer, loss="binary_crossentropy")
siamese_net.count_params()

23111489

In [4]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_6
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_2
20 dense_1


In [31]:
class0=os.path.join(os.getcwd() +'/face/train_split_rgb/0/' )
class1=os.path.join(os.getcwd() +'/face/train_split_rgb/1/' )
class2=os.path.join(os.getcwd() +'/face/train_split_rgb/2/' )
class3=os.path.join(os.getcwd() +'/face/train_split_rgb/3/' )
class4=os.path.join(os.getcwd() +'/face/train_split_rgb/4/' )
class5=os.path.join(os.getcwd() +'/face/train_split_rgb/5/' )
class6=os.path.join(os.getcwd() +'/face/train_split_rgb/6/' )
#50% same 50% different class ,randomly sampled pairs.
#Ensured 6 images from each class used in training.
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = scipy.misc.imread(os.path.join(folder,filename))
        images.append(img)
    return images
Train_image=[load_images(class0),load_images(class1),load_images(class2),load_images(class3), \
            load_images(class4),load_images(class5),load_images(class6)]

"""Create batch of 42 training pairs, 21 pairs with same class, 21 pairs with different class"""
batch_size=42
w, h = 64,64
pairs=[np.zeros((batch_size, w, h,3)) for i in range(2)]
twin1_catagories_1=np.array([0,0,0,1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6])
catagory_11=twin1_catagories_1  #for simplification purposes
twin1_catagories_2=np.array([5, 1, 1, 0, 0, 4, 6, 3, 4, 6, 4, 2, 2, 2, 5, 3, 6, 3, 0, 1, 5])
catagory_12=twin1_catagories_2  #for simplification purposes
twin2_catagories=twin1_catagories_1
"np.random.seed(10)"
"idx=np.random.choice(6000,42,replace=False)"
"Code commented are implemented separately to avoid fixed seed for random processes in future codes."

idx=[1789, 5566, 1657, 3983, 5619, 1933,  449,  492, 1172, 2040, 1638,
2864,   43, 5540, 3758, 5606, 4936, 3399, 5436, 2602,  708, 4879,
5593, 4736, 1324,  952, 3036, 4971, 2630, 2543, 4627, 1044,  502,
1477, 5107, 4155, 2501, 3975,  143, 2083, 2514, 2569]
b=list(np.arange(0,6000))
for i in idx:
    b.remove(i)


"Since we are only using 42 pairs of photos, we can treat remaning entire training images as validation set. There is now"
"two separate dataset for training and validation."


def load_batch_train_pairs(batch_size):
    targets=np.zeros((batch_size,))
    targets[batch_size//2:] = 1
    for i in range(42):
        if i<batch_size//2:
            pairs[0][i,:,:,:]=Train_image[catagory_11[i]][idx[i]]
            pairs[1][i,:,:,:]=Train_image[catagory_12[i]][idx[i]]
        if i>=batch_size//2:
            pairs[0][i,:,:,:]=Train_image[catagory_11[i-(batch_size//2)]][idx[i]] 
            pairs[1][i,:,:,:]=Train_image[catagory_11[i-(batch_size//2)]][idx[i-(batch_size//2)]]
    return pairs,targets
def get_one_shot(N):
    "N = no.of class/N classes to distinguish from"
    categories = rng.choice(range(7),size=(N,),replace=False)            
    true_category = categories[0]
    index=rng.choice(b,replace=False,size=(N,))
    ex1, ex2 = rng.choice(b,replace=False,size=(2,))
    test_image_set = np.asarray([Train_image[true_category][ex1]]*N)
    support_set=np.zeros((N,64,64,3))
    for i in range(1,N):
        support_set[i,:,:,:] = Train_image[categories[i]][index[i]]
        support_set[0,:,:,:] = Train_image[true_category][ex2]
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image_set, support_set = shuffle(targets, test_image_set, support_set) #Shuffle and give the same random orders of 
    #correct & incorrect pairs in three sets. 
    pairs = [test_image_set,support_set]
    return pairs,targets
def oneshot_accuracy(model,k,N,verbose=0):
    "Calculation for accuracy of one-shot task to validation/test set after training the model"
    n_correct=0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ...".format(k,N))
    for i in range(k):
        inputs,targets=get_one_shot(N)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
                n_correct+=1
    percent_correct = (100.0*n_correct / k)
    if verbose:
        print("Got an average of {}% over {} random {} way few-shot learning accuracy".format(percent_correct,k,N))
    return percent_correct

In [47]:
"Training Siamese network:"
siamese_net.fit(x=load_batch_train_pairs(42)[0],y=load_batch_train_pairs(42)[1],batch_size=42,epochs=10,validation_data=None,verbose=1)
#for i in range(epochs):
#    val_acc = oneshot_accuracy(siamese_net,N_way,n_val,verbose=True)
#val_acc

Epoch 1/10
42/42 [==============================] - 6s 151ms/step - loss: 6710.5049
Epoch 2/10
42/42 [==============================] - 6s 139ms/step - loss: 6710.4990
Epoch 3/10
42/42 [==============================] - 6s 138ms/step - loss: 6710.4941
Epoch 4/10
42/42 [==============================] - 6s 140ms/step - loss: 6710.4883
Epoch 5/10
42/42 [==============================] - 6s 138ms/step - loss: 6710.4839
Epoch 6/10
42/42 [==============================] - 6s 140ms/step - loss: 6710.4795
Epoch 7/10
42/42 [==============================] - 6s 150ms/step - loss: 6710.4746
Epoch 8/10
42/42 [==============================] - 6s 144ms/step - loss: 6710.4707
Epoch 9/10
42/42 [==============================] - 7s 155ms/step - loss: 6710.4668
Epoch 10/10
42/42 [==============================] - 6s 152ms/step - loss: 6710.4629


In [50]:
val_acc = oneshot_accuracy(siamese_net,200,7,verbose=True)
print(val_acc)

Evaluating model on 200 random 7 way one-shot learning tasks ...
Got an average of 16.5% over 200 random 7 way few-shot learning accuracy
16.5


In [101]:
b=np.array([0,0,0,1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6])
c=np.array([0,0,0,1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6])
d=np.array([5, 1, 1, 0, 0, 4, 6, 3, 4, 6, 4, 2, 2, 2, 5, 3, 6, 3, 0, 1, 5])
for i in range(21):
    while c[i] == b[i]:
        np.random.shuffle(c)
c
#targets=np.zeros((42,))
#targets[21//2:] = 1
#targets
b==d

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False], dtype=bool)

In [31]:
n_iterations
for i in range(1, n_iter):
    (inputs,targets)=train_pairs(batch_size)
    loss=model.train_on_batch(inputs,targets)
    print(loss)

array([3, 6])

In [65]:
np.random.seed(10)
np.random.choice(6000,42,replace=False)

array([1789, 5566, 1657, 3983, 5619, 1933,  449,  492, 1172, 2040, 1638,
       2864,   43, 5540, 3758, 5606, 4936, 3399, 5436, 2602,  708, 4879,
       5593, 4736, 1324,  952, 3036, 4971, 2630, 2543, 4627, 1044,  502,
       1477, 5107, 4155, 2501, 3975,  143, 2083, 2514, 2569])

In [11]:
for i in range(42):
    if i>=42//2:
        print(batch_size//2)i)

SyntaxError: invalid syntax (<ipython-input-11-1d24b58863eb>, line 3)